In [1]:
# # the imports in this cell are required when running on local device
# import os, sys
# sys.path.append(os.path.join('..', '..'))
# from utils.applyML_util import train_regression, eval_regression
# from utils.featureSelection_util import (pearson_correlation_fs, 
#                                          seleckKBest_fs, selectSequential_fs)

In [2]:
# the imports in this cell are required when running from Cloud (Colab/Kaggle)
# before running on cloud you nee to upload the .py files
# from 'Notebooks/utils' directory
from applyML_util import train_regression, eval_regression, showEvalutationGraph_regression
from featureSelection_util import (pearson_correlation_fs, 
                                   seleckKBest_fs, selectSequential_fs)

# SVC & SVR with Minimized Hyperparameters


**SVR Documentation link:** https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html <br>
**SVC Documentation link:** https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html <br>
**SK-Lego Documentation link:** https://scikit-lego.netlify.app/meta.html#Zero-Inflated-Regressor

In [ ]:
!pip install sklego

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklego.meta import ZeroInflatedRegressor #!pip install sklego
from sklearn.svm import SVR
from sklearn.svm import SVC

In [5]:
# global random seed
RAND_SEED = 42

ziReg = ZeroInflatedRegressor(classifier=SVC(), regressor=SVR())

# initial model with only random seed and not any hyper-parametes
initial_model = ziReg

# hyper-parameters
regressor__kernel = ['poly']
regressor__degree = [5]
regressor__epsilon = [0.5]
regressor__C = [6,7,8,9]

classifier__kernel = ['poly']
classifier__C = regressor__C
classifier__degree = [5]
classifier__gamma = [1,0.1]


param_grid = {'classifier__kernel':classifier__kernel,'classifier__gamma':classifier__gamma,'classifier__degree': classifier__degree,'classifier__C':classifier__C,'regressor__kernel': regressor__kernel,'regressor__C': regressor__C, 'regressor__epsilon': regressor__epsilon, 'regressor__degree': regressor__degree}

# variables needed for showEvalGraph_regression() function
MODEL_CLASS = ziReg
x_axis_param_name = 'regressor__C' #???classifier__C
x_axis_vals = regressor__C

In [6]:
ziReg.get_params().keys()


dict_keys(['classifier__C', 'classifier__break_ties', 'classifier__cache_size', 'classifier__class_weight', 'classifier__coef0', 'classifier__decision_function_shape', 'classifier__degree', 'classifier__gamma', 'classifier__kernel', 'classifier__max_iter', 'classifier__probability', 'classifier__random_state', 'classifier__shrinking', 'classifier__tol', 'classifier__verbose', 'classifier', 'regressor__C', 'regressor__cache_size', 'regressor__coef0', 'regressor__degree', 'regressor__epsilon', 'regressor__gamma', 'regressor__kernel', 'regressor__max_iter', 'regressor__shrinking', 'regressor__tol', 'regressor__verbose', 'regressor'])

## 1. Experimentation on the Weather Daily dataset

In [7]:
# Load the train dataset
weather_daily_train_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/train/brri-weather_train_regression.csv')

# Load the test set
weather_daily_test_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/test/brri-weather_test_regression.csv')

In [8]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 8, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__C': 7, 'regressor__degree': 5, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.1909, mae=5.2048, rmse=13.4731


In [9]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [10]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1359, mae=5.4906, rmse=16.0731


### 1.1 Apply Pearson Feature Selection to Daily Weather Dataset

In [11]:
# select features from the train dataset
weather_daily_fs1_train_df, cols_to_drop = pearson_correlation_fs(weather_daily_train_df, 'Rainfall (mm)')

# keep only selected features on the test dataset
weather_daily_fs1_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

dropping Sunshine (hour/day) from (Cloudy (hour/day), Sunshine (hour/day))


In [12]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs1_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 8, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__C': 9, 'regressor__degree': 5, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.2021, mae=5.1403, rmse=13.3844


In [13]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs1_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [14]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs1_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1487, mae=5.3964, rmse=15.9535


### 1.2 Apply SelectKBest Feature Selection to Daily Weather Dataset

In [15]:
# select features from the train dataset
weather_daily_fs2_train_df, cols_to_drop = seleckKBest_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs2_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Max Temp. (degree Celcius)', 'Solar Radiation (cal/cm^2/day)']


In [16]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs2_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 8, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__C': 7, 'regressor__degree': 5, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.1818, mae=5.1461, rmse=13.5553


In [17]:
# r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs2_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [18]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs2_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.148, mae=5.2718, rmse=15.9603


### 1.3 Apply SelectSequential Feature Selection to Daily Weather Dataset

In [19]:
# select features from the train dataset
weather_daily_fs3_train_df, cols_to_drop = selectSequential_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs3_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Actual Evaporation (mm)', 'Relative Humidity (morning, %)']


In [20]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs3_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 7, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__C': 9, 'regressor__degree': 5, 'regressor__epsilon': 0.5, 'regressor__kernel': 'poly'}
Train set performance: r2-score=0.1901, mae=5.1796, rmse=13.489


In [21]:
# r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs3_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [22]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs3_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1297, mae=5.4658, rmse=16.1304
